In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.utils import np_utils
from keras.utils import plot_model
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.
C:\Users\ganta\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ganta\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ganta\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ganta\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
# Read file - wonderland.txt
filename = 'wonderland.txt'
data = open(filename, 'r')
data = data.read().lower()

In [3]:
print("Length of the data: "+ str(len(data))) 

Length of the data: 12142


In [7]:
# data

In [9]:
# list(set(data))

In [4]:
# Find unique characters in the data
chars = sorted(list(set(data)))
print("Number of unique charaters in the data: " + str(len(chars)))

Number of unique charaters in the data: 51


In [5]:
chars

['\n',
 ' ',
 '!',
 '"',
 '#',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '8',
 '9',
 ':',
 ';',
 '?',
 '[',
 ']',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [40]:
# chars

In [14]:
# Map characters to integers
char_to_integer = []
for integer, char in enumerate(chars):
    char_to_integer.append((char, integer))
char_to_integer = dict(char_to_integer)
print(char_to_integer)


{'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, "'": 5, '(': 6, ')': 7, '*': 8, ',': 9, '-': 10, '.': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '8': 18, '9': 19, ':': 20, ';': 21, '?': 22, '[': 23, ']': 24, 'a': 25, 'b': 26, 'c': 27, 'd': 28, 'e': 29, 'f': 30, 'g': 31, 'h': 32, 'i': 33, 'j': 34, 'k': 35, 'l': 36, 'm': 37, 'n': 38, 'o': 39, 'p': 40, 'q': 41, 'r': 42, 's': 43, 't': 44, 'u': 45, 'v': 46, 'w': 47, 'x': 48, 'y': 49, 'z': 50}


In [31]:
# Convert X(sequence) and y(one character) into integers
X_train = []
y_train = []
length_of_sequence = 100
for i in range(0, (len(data) - length_of_sequence)):
    sequence = data[i:i+length_of_sequence]
    sequence_int = []
    for char in sequence:
        sequence_int.append(char_to_integer[char])
    X_train.append(sequence_int)    
    label = data[i+length_of_sequence]
    y_train.append(char_to_integer[label])

In [23]:
# Reshape X_train and normalize
samples = len(X_train)
X_train = np.reshape(X_train, (samples, length_of_sequence, 1)) 
X_train = X_train/float(len(chars))
print(X_train.shape)

(12042, 100, 1)


In [33]:
# y_train: Integers to one hot vectors
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)



(12042, 51)


In [34]:
y_train

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [20]:
X_train.shape

(12042, 100, 1)

In [23]:
X_train.shape[1]

100

In [9]:
# Model
model = Sequential()
model.add(LSTM(256, input_shape = (X_train.shape[1], X_train.shape[2]))) # 256 hidden nodes
model.add(Dropout(0.2))#This technique is applied in the training phase to reduce overfitting effects.# https://www.python-course.eu/neural_networks_with_dropout.php
model.add(Dense(y_train.shape[1], activation = 'softmax'))
model.summary()

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X_train, y_train, epochs = 4, batch_size = 32)
model.save('model_weights_LSTM.h5')

# plot_model(model, to_file='model_LSTM.png', show_shapes=True, show_layer_names=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 51)                13107     
Total params: 277,299
Trainable params: 277,299
Non-trainable params: 0
_________________________________________________________________
Epoch 1/4
12042/12042 [==============================] - 74s 6ms/step - loss: 0.0792 - acc: 0.9804
Epoch 2/4
12042/12042 [==============================] - 74s 6ms/step - loss: 0.0782 - acc: 0.9804: 
Epoch 3/4
12042/12042 [==============================] - 79s 7ms/step - loss: 0.0775 - acc: 0.9804
Epoch 4/4
12042/12042 [==============================] - 86s 7ms/step - loss: 0.0754 - acc: 0.9804
